In [ ]:
import numpy as np
import cv2
import scipy
import matplotlib.pylab as plt
from skimage import io

vid = cv2.VideoCapture('videos/stereo/left_output.avi')
template_img = cv2.imread('template-1.png')
template_gray = cv2.cvtColor(template_img, cv2.COLOR_BGR2GRAY)
dX=0
dY=0
dZ=0
counter = 0
width  = 0   # float width
height = 0  # float height
vcap=vid
if vcap.isOpened(): 
    width  = vcap.get(3)  # float width
    height = vcap.get(4)  # float height

middle_w=width/2
middle_h=height/2
print(middle_w)
print(middle_h)


while vid.isOpened() :
    ret, query_img = vid.read()
    ret, img = vid.read()
    query_gray = cv2.cvtColor(query_img, cv2.COLOR_BGR2GRAY)
    if ret :
        sift = cv2.SIFT_create()
        bf = cv2.BFMatcher()
        template_kpts, template_desc = sift.detectAndCompute(template_gray, None)
        query_kpts, query_desc = sift.detectAndCompute(query_gray, None)
        matches = bf.knnMatch(template_desc, query_desc, k=2)

        
        good_matches = list()
        good_matches_list = list()
        for m, n in matches :
            if m.distance < 0.6*n.distance :
                good_matches.append(m)
                good_matches_list.append([m])
        
        if len(good_matches) > 10 :
            src_pts = np.float32([ template_kpts[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
            dst_pts = np.float32([ query_kpts[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)

            H, inlier_masks = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0) # H RANSAC
            # get the bounding box around template image
            h, w = template_img.shape[:2]
            template_box = np.float32([[0, 0], [0, h], [w, h], [w, 0]]).reshape(-1,1,2)
            transformed_box = cv2.perspectiveTransform(template_box, H)
            detected_img = cv2.polylines(query_img, [np.int32(transformed_box)], True, (0,255,0), 3, cv2.LINE_AA)
            dX = transformed_box[0][0]
            dY = transformed_box[1][0]
            dZ = 0
            poptb1=transformed_box[0]
            poptb2=transformed_box[1]
            popsum=poptb2[0]-poptb1[0]
           
            
            cv2.putText(query_img, "X: {}, Y: {}, Z: {}".format(int(dX[0]-middle_w),int(dY[1]-middle_h) , int(popsum[1])),
		(10,query_img.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX,
		1, (0, 255, 0), 2)
            
            # cv2.imshow('Video frame', detected_img)
            cv2.imshow('Video frame', detected_img)
          

        if cv2.waitKey(int(1000/24)) & 0xFF == ord('q') : # this line control the period between image frame
            break
    else :
        break
vid.release()
cv2.destroyAllWindows()